# Load Functions and Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.fft import fft, fftfreq
import itertools
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.metrics import auc
from tkinter.filedialog import askdirectory
import tkinter as tk
from ipywidgets import interact, widgets, fixed, interactive, HBox, Layout
import pandas as pd
from peakutils import baseline
import os 
from tkinter import simpledialog
from operator import contains
import pickle
import pybaselines
from matplotlib import colormaps
import matplotlib as mpl

# Custom Functions
from FunctionsProcessMulti_Longit import getDirsAll, getWellNum, extrMDAll, getTimeVec, getLB, ProcDatMult_Long, plotDatGen, interactPhase, appPhsBsl, interStack3D, interStack2D, genSumUpDat, interSumUpSp, getFWHM, getDatFuture, getSNR, interactIntegrals, getIntegrals, interIntegrals, interIntegralsRatios, procDatGen, shiftPPM

# Select directory with the data and extract subexperiments folders automatically

In [18]:
test, fldslst3, dircns, lngrps, lgtdt, stck, machi = getDirsAll(r"C:\IBECPostDocDrive\2025_04_14_BIST_FertEyeExperiments\BIST_MRI")

EXPERIMENT PROCESSED INFORMATION:
---------------------------------


The following directory contains 1 subexperiments. 

Each sub experiment contains 1 sequential Slabs. 

The subexperiment 1 has 50 scans per slab. 

In the last processing, you selected an apodisation factor of 4.0. 

In the last processing, you selected 1 to 10 scans for the sum up final spectra. 



 

User Comments: 

Seems good


# Process Data

In [ ]:
numWells = 1 # This is for more complex acquisitions with multiple regions. If so, change to the number of slabs done. 
metDat, indx, foldPath = extrMDAll(lngrps, test, lgtdt, machi) # Function to extract necessary metadata for each single pulse scan folder for each subexperiment and each well
timVecs = getTimeVec(lngrps, numWells, lgtdt, metDat, machi) # Function to extract and compute the time vector for each time well and for each sub-experiment
lineborad = getLB() # Ask user the factor desired for decaying exponential apodisation
ppms, datFutmag, datFut, datco3, foldsDis = procDatGen(lgtdt, lngrps, metDat, numWells, foldPath, lineborad, machi, stck = stck,  xin = 76) # Function to extract the raw spectroscopic data, rearrange it into an FID, do MRI phase corrections and generate a fourier transform

#### Plot FID

In [6]:
plotDatGen(ppms, foldsDis, datco3, datFutmag, datFut, foldPath, metDat, machi, datyp = "FID")

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

#### Plot magnitude spectra 

In [76]:
plotDatGen(ppms, foldsDis, datco3, datFutmag, datFut, foldPath, datyp = "mag")

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

#### Plot non-phased spectra

In [24]:
plotDatGen(ppms, foldsDis, datco3, datFutmag, datFut, foldPath, datyp = "real")

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

# Manual Phase Correction
CAREFULL!!! Go spectra by spectra, but do not go back to one that is already phased or phase values will be overwriten. This will be fixed in latter releases

In [21]:
interactPhase(foldPath, datFut) # This are the necessary functions to interactively do phase correction for all sub experiments and slab selections. 

#### Visualise Corrected Data

In [22]:
phasePars, datFutPh, datFutPhBSl, datBaselines = appPhsBsl(foldPath, datFut, ppms, rdbc=False) # This function applyes the phase correction (and baseline correction) to your dataset iteratively. Set last parameter to true to re-do the baseline correction
plotDatGen(ppms, foldsDis, datco3, datFutmag, datFutPhBSl, foldPath, datyp = "phsd")

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

# Plot Spectra Stacks

#### 3D

In [ ]:
clls = ["#80000aff", "Red", "Green", "Blue", "Yellow", "Cyan", "Magenta", "Black", "Grey"] # Add or remove any colour choices you desire for the plot.
interStack3D(foldPath, datFutPhBSl, ppms, clls=clls, grad=False) # Functions to plot a stack of the spectra acquired in a 3D plot for better visualisation. Users can move the plot around, change colors and all the functions from the previous plots. 

# David: Change scan axis for time???


interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

#### 2D

In [55]:
clls = ["#80000aff", "Red", "Green", "Blue", "Yellow", "Cyan", "Magenta", "Black", "Grey"] # Add or remove any colour choices you desire for the plot.
interStack2D(foldPath, datFutPhBSl, ppms, clls = clls) # Functions to plot a stack of the spectra acquired in a 2D plot (MNova Style) for better visualisation. Users can move the plot around, change colors and all the functions from the previous plots. 

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

# Generate and Plot Sum Up Spectra

In [23]:
datFutPhBSl_Sum, SUS = genSumUpDat(foldPath, datFutPhBSl)
clls = ["#db6030ff", "#80000aff", "Red", "Green", "Blue", "Yellow", "Cyan", "Magenta", "Black", "Grey"]
interSumUpSp(foldPath, datFutPhBSl_Sum, ppms, clls = clls) # Functions to visualise the sum up spectra in an interactive plot

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

# Check Any Spectra (Including Sum Up) Bandwidth

In [24]:
getFWHM(foldPath, datFutPhBSl, datFutPhBSl_Sum, ppms, metDat, machi) # Functions to display spectra to estimate the width of any desire peak at any point

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

# Check Any Spectra (Including Sum Up) SNR

In [ ]:
getSNR(foldPath, datFutPhBSl, datFutPhBSl_Sum, ppms)

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1, 2), value=1), Dropdown(description='Slab:'…

# Integration

In [95]:
interactIntegrals(foldPath, datFutPhBSl, lngrps, numWells, ppms)

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

#### Plot Integration Over Time

In [ ]:
integVals, integPPMS, integInte, intPars = getIntegrals(foldPath, datFutPhBSl, ppms) # Function to extract the integration regions assigned before and compute the AUC for each. 
interIntegrals(foldPath, datco3, datFutPhBSl, integVals, timVecs) # Function to interactively display the integration resutls for all subexperiments

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

#### Plot Integration Ratios

In [ ]:
interIntegralsRatios(foldPath, datco3, datFutPhBSl, integVals, timVecs, SUS) # Functions to interactively visualise AUC of data and ratios

interactive(children=(Dropdown(description='Sub.Exp.:', options=(1,), value=1), Dropdown(description='Slab:', …

# Add a note file to remind user of things for pre-processed data. 

In [86]:
getDatFuture(lngrps, numWells, foldsDis, lineborad, SUS, foldPath) # Function to extract important info about the processing to remind the user the next time it tries to process it again. It also asks the user if they want to add any additional comment for the future. Careful since every time you run this function, the previous file will be overwritten. 